In [ ]:
import numpy as np
import pandas as pd
from netCDF4 import Dataset
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:
import matplotlib.colors as mcolors
to_rgb = mcolors.ColorConverter().to_rgb
# d = {
#     0.0: 'blue', 
#     0.2: 'cyan', 
#     0.4: 'yellow', 
#     0.5: 'orange', 
#     0.7: 'red', 
#     0.9: (0.5, 0, 0),
#     1.0: (0.4, 0, 0)
# }
d = {
    0.0: (0.4, 0, 0),
    0.2: (0.5, 0, 0),
    0.4: 'red',
    0.5: 'orange',
    0.7: 'yellow',
    0.9: 'cyan',
    1.0: "blue"
}

def create_cmap(points):
    red, green, blue = [], [], []
    for idx, color in points.items():
        r, g, b = to_rgb(color) if type(color)==str else color
        red.append([idx, r, r])
        green.append([idx, g, g])
        blue.append([idx, b, b])
    red.append([1,0.2,0])
    green.append([1,0,0])
    blue.append([1,0,0])
    
    return { 'red': red, 'green': green, 'blue': blue}

newcmp = LinearSegmentedColormap('testCmap', segmentdata=create_cmap(d), N=256)
newcmp

In [ ]:
# Read dataset
dataset = Dataset(r'Mercator_20210315_20210318.nc')
dataset

In [ ]:
import os
os.getcwd()

In [ ]:
# Get time data
time = np.array(dataset.variables['time'])
time

In [ ]:
# Get depth data
depth = np.array(dataset.variables['depth'][:])
depth

In [ ]:
# Get latitide data
latitude = np.array(dataset.variables['latitude'])
latitude

In [ ]:
# Get longitude data
longitude = np.array(dataset.variables['longitude'])
longitude

## Temperatura

In [ ]:
# Get temperature data from specific time and depth
temp_data = dataset.variables['thetao'][:]
temp = temp_data[0][0]
temp

In [ ]:
# Temperatura
df = pd.DataFrame(temp, index=latitude, columns=longitude)
df = df.sort_index(ascending=False)

txt = 'var temperature = ['
for row, row_data in df.iterrows():
    for col, data in row_data.iteritems():
        if (not pd.isnull(data)):
            txt += '[%f, %f, %f],\n' % (row, col, data)
txt += ']'

with open('temperature.js', 'w') as f:
    f.write(txt)

plt.title('Temperature')
sns.heatmap(df, cmap=newcmp)

## Salinidade

In [ ]:
# Get temperature data from specific time and depth
sal_data = dataset.variables['so'][:]
sal = sal_data[0][0]
sal

In [ ]:
# Salinidade
df = pd.DataFrame(sal, index=latitude, columns=longitude)
df = df.sort_index(ascending=False)

txt = 'var salinity = ['
for row, row_data in df.iterrows():
    for col, data in row_data.iteritems():
        if (not pd.isnull(data)):
            txt += '[%f, %f, %f],\n' % (row, col, data)
txt += ']'

with open('salinity.js', 'w') as f:
    f.write(txt)

plt.title('Salinity')
sns.heatmap(df, cmap=newcmp)

## Wind

In [ ]:
# Get u
u_data = dataset.variables['uo'][:]
u = u_data[0][0]
u

In [ ]:
# U
dfu = pd.DataFrame(u, index=latitude, columns=longitude)
dfu = dfu.sort_index(ascending=False)
# dfu = dfv.fillna(-99)
dfu

In [ ]:
# Get v
v_data = dataset.variables['vo'][:]
v = v_data[0][0]
v

In [ ]:
# V
dfv = pd.DataFrame(v, index=latitude, columns=longitude)
dfv = dfv.sort_index(ascending=False)
# dfv = dfv.fillna(-99)
dfv

In [ ]:
x = df.columns.values
y = df.index.values

fig, ax = plt.subplots()
q = ax.quiver(x, y, dfu.values, dfv.values)
ax.quiverkey(q, X=0.3, Y=1.1, U=0.5,
             label='Quiver key, length = 0.5', labelpos='E')

In [ ]:
import json
# Criar arquivo de saida
nx = len(x)
ny = len(y)
dx = float(np.diff(longitude).mean())
dy = float(np.diff(latitude).mean())
tot = nx*ny
la1,la2 = float(max(latitude)), float(min(latitude))
lo1,lo2 = float(min(longitude)), float(max(longitude))
udata = np.nan_to_num(dfu.values.flatten()*10)
vdata = np.nan_to_num(dfv.values.flatten()*10)

wind_data = [{
  "header": {
    "parameterNumberName": "eastward_wind",
    "parameterUnit": "m.s-1",
    "parameterNumber": 2,
    "parameterCategory": 2,
    "nx": nx,
    "ny": ny,
    "numberPoints": tot,
    "dx": dx,
    "dy": dy,
    "la1": la1,
    "lo1": lo1,
    "la2": la2,
    "lo2": lo2,
#     "refTime": "2017-02-01 23:00:00"
  },
  "data": list(udata)
}, {
  "header": {
    "parameterNumberName": "northward_wind",
    "parameterUnit": "m.s-1",
    "parameterNumber": 3,
    "parameterCategory": 2,
    "nx": nx,
    "ny": ny,
    "numberPoints": tot,
    "dx": dx,
    "dy": dy,
    "la1": la1,
    "lo1": lo1,
    "la2": la2,
    "lo2": lo2,
#     "refTime": "2017-02-01 23:00:00"
  },
  "data": list(vdata)
}]

with open('wind.json', 'w') as outfile:
    json.dump(wind_data, outfile, separators=(',', ':'), indent=4)